In [1]:
from pyspark.sql import SparkSession
import pyspark.pandas as ps

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
spark = SparkSession.builder.getOrCreate()
ps.set_option('compute.ops_on_diff_frames', True)

In [ ]:
data=spark.read.options(header=True,inferSchema=True).csv('sample.csv')
data

In [ ]:
pandas_data=data.pandas_api()
pandas_data

In [ ]:
pandas_data=pandas_data.filter(axis=0,items=pandas_data.applicationid.dropna().index.to_list())
pandas_data

In [ ]:
pandas_data=ps.concat([pandas_data,pandas_data.index.to_frame()],axis=1).sort_values(by=0,ascending=False).drop([0],axis=1).reset_index().drop(['index','level_0'],axis=1)
pandas_data

In [ ]:
grouped_data=pandas_data.groupby([(pandas_data.saallocationstart != pandas_data.saallocationstart.shift()).cumsum(),'keydatasource'])

In [ ]:
indices=grouped_data.xmltimestamp.idxmax()
indices

In [ ]:
snapshot_count=grouped_data.snapshotid.count()
snapshot_count

In [ ]:
mappings=indices.to_dataframe().reset_index().merge(snapshot_count.to_dataframe().reset_index()).drop(['saallocationstart','keydatasource'],axis=1).rename(columns={'snapshotid':'snapshot_count'})
mappings

In [ ]:
final_data=pandas_data.iloc[indices.to_list()].reset_index()
final_data

In [ ]:
final_data=final_data.merge(mappings,left_on='index',right_on='xmltimestamp',suffixes=('','_y')).drop(['xmltimestamp_y','index'],axis=1)
final_data

In [ ]:
final_data=final_data[final_data['snapshot_count']>1]
final_data

In [ ]:
raw_data['Mycol'] =  pd.to_datetime(raw_data['Mycol'],

In [ ]:
final_data.to_csv(path='output.csv')

In [ ]:
df.to_spark(index_col="index").show()  